In [1]:
import math
import random
from torchvision import datasets ,transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset
from torch import nn 
from torch.nn.functional import relu ,softmax 
import copy 
from torch.utils.data import Subset
import pandas as pd 
import numpy as np
from PIL import Image
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
#device=torch.device("cuda:0"if torch.cuda.is_available () else "cpu")
device="cpu"

In [2]:
import pandas as pd

path ='Riktim_Testing/Word-level-Features/MLG_180_Features.csv'
df = pd.read_csv(path) #header=None
df.head()

,att_1,att_2,att_3,att_4,att_5,att_6,att_7,att_8,att_9,att_10,...,att_172,att_173,att_174,att_175,att_176,att_177,att_178,att_179,att_180,class
0,12.184764,18.534243,28.945810,22.241393,7.353465,17.095399,26.996429,41.427220,24.394928,3.888953,...,0.010558,0.014658,0.016326,0.009556,0.007948,0.011146,0.016568,0.017170,0.008511,A
1,25.639720,46.945876,63.464762,44.700204,23.165613,18.646121,28.515218,41.926007,29.490375,8.800943,...,0.014752,0.022414,0.024294,0.014430,0.010937,0.016972,0.025050,0.023918,0.014216,A
2,14.381637,24.706305,35.822925,29.553382,22.935236,13.837493,22.689212,36.892506,33.376486,9.703304,...,0.013782,0.019098,0.024171,0.020194,0.009770,0.014811,0.021661,0.021909,0.015823,A
3,20.123721,39.021980,59.427789,33.768032,16.884216,14.235512,24.346061,33.150834,17.558946,5.433499,...,0.014727,0.022686,0.022876,0.013031,0.010639,0.016706,0.026014,0.023591,0.012991,A
4,20.855774,31.241891,46.056155,36.617243,20.152911,15.612698,27.206975,42.754484,29.983009,7.457517,...,0.014497,0.019523,0.023203,0.016096,0.011517,0.016587,0.024086,0.022221,0.015074,A


In [3]:
print(df.shape)

(12000, 181)


In [4]:
print(df.shape)
#X=df.loc[:,df.columns != 'Classifier'].values
X=df.loc[:,df.columns != df.columns[-1]].values
print(type(X))
print(X.shape)
#y_label=df['Classifier'].values 
y_label =df.iloc[:,-1].values
#print(type(y_label))
#print(X[0])
print()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_label)
print(le.classes_)
y=le.transform(y_label)
print(y[:10])

(12000, 181)
<class 'numpy.ndarray'>
(12000, 180)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L']
[0 0 0 0 0 0 0 0 0 0]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler #for normalization
sc = StandardScaler()
X = sc.fit_transform(X)
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(0, 1))
# X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
# print(y_train)
# print(y_test)
#print(len(X_train), len(y_train), len(X_test), len(y_test))
# # X=X_train
# # y=y_train
print(set(y_train), set(y_test))
print(X.shape, y.shape)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11} {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
(12000, 180) (12000,)


In [6]:
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

In [7]:
#n_inputs = X_train.shape[1]
#n_hidden = 180
#n_classes = len(le.classes_)
#N = X_train.shape[0] # Number of samples

#print(N)
#hyperparameters
input_size = X_train.shape[1]    # The image size = 28 x 28 = 784
hidden_size = X_train.shape[1]    # The number of nodes at the hidden layer
num_classes = len(le.classes_)       # The number of output classes. In this case, from 0 to 9
num_epochs = 100      # The number of times entire dataset is trained
batch_size = X_train.shape[0]    # The size of input data took for one iteration
lr = 0.001  # The speed of convergence

In [8]:
# #build model
# class Net(nn.Module):

#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(n_inputs, n_hidden)
#         self.drop = nn.Dropout (p=0.5)
#         self.fc2 = nn.Linear(n_hidden,n_classes )

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         #x=self.drop(x)
#         x = self.fc2(x)
#         return x
        
# net = Net()
class Model(nn.Module):
    def __init__(self,input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size , hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.drop = nn.Dropout (p=0.4)

    def masking (self,act1,mask,p):
        if (self.training ==True ):
            return ((act1*mask)/p)
        else :
            return (act1)
        

    def Forward(self, x,mask,p):
        #print(x.shape)
        #x = x.view(-1, 32 * 32 * 3)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        #x = self.drop(x)
        x = self.masking (x,mask,p)
        x = self.fc2(x)
        return x

In [9]:
def train_model(model, criterion, optimizer,mask,p, num_epochs=25):
  # the mask has to be specified 
      since = time.time()

      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      losses=[]
      accuracies=[]

      for epoch in range(num_epochs):
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)

          # Each epoch has a training and validation phase
          for phase in ['train', 'val']:
              if phase == 'train':
                  model.train()  # Set model to training mode
                  optimizer.zero_grad()
                  with torch.set_grad_enabled(phase == 'train'):
                    outputs= model.Forward(X_train,mask,p) #inputs
                    train_loss = criterion(outputs, y_train)# backward + optimize only if in training phase,labels
                    train_loss.backward()
                    optimizer.step()
                  train_acc = 100 * torch.sum(y_train==torch.max(outputs.data, 1)[1]).double() / len(y_train)
                  print('********************{}*************'.format(phase))
                  print ('Epoch [%d/%d] Loss: %.4f   Acc: %.4f'%(epoch+1, num_epochs, train_loss.item(), train_acc.item()))
                  losses.append(train_loss)
                  accuracies.append(train_acc)
              else:
                  model.eval()   # Set model to evaluate mode
                  outputs= model.Forward(X_test,mask,p) #inputs
                  _, predicted = torch.max(outputs, 1)
                  test_loss = criterion(outputs, y_test)
                  test_acc = 100 * torch.sum(y_test==predicted).double() / len(y_test)
                  print('********************{}*************'.format(phase))
                  print ('Epoch [%d/%d] Loss: %.4f   Acc: %.4f'%(epoch+1, num_epochs, test_loss.item(), test_acc.item()))
                  if test_acc > best_acc:
                    best_acc = test_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

              



      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
          time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f}'.format(best_acc))

      # load best model weights
      model.load_state_dict(best_model_wts)
      return model,losses,accuracies

In [10]:
class Population:   
    def __init__(self,m,num,maskLength):
        # constructor for initialising the population list
        #list of DNA objects
        self.population=[]
        #muation rate for mutation
        self.mutation_rate=m
        #maximum number of entities in the population
        self.popmax=num
        self.maskLength=maskLength
        for i in range (num):
            #creating a dna object
            #an initial random population created 
            dna =DNA(self.maskLength)
            self.population.append (dna)
      
        self.matingPool=[]

    def calcFitness (self,model):
        # going through all the entities of population 
        #finding fitness of all population entities 
        for i in range(0,self.popmax):
            self.population[i].fitness (model)

    def naturalSelection(self):
        self.matingPool=[]
        maxFitness=0
        for i in range (self.popmax):
            # moving throught the entire population 
            if (self.population[i].fit>maxFitness):
                maxFitness=self.population[i].fit
       
        # max Fitness has the maximum loss score of the entire population  
        for i in range (self.popmax ):
        # iterating through the all inviduals of the population
            n=self.Mymap(self.population[i].fit,0,maxFitness,0,1)
        
            n=math.floor(n*100)
            
            for j in range (n):
                #creating mating pool
                self.matingPool.append (self.population[i])

    def Mymap(self,num,prevlow,prevhigh,nextlow,nexthigh):
        
        prevrange =float((num-prevlow)/(prevhigh-prevlow))
        return nextlow+(nexthigh-nextlow)*prevrange

    def   generate (self):
        for i in range (self.popmax ):
            index_1=math.floor(random.randint  (0,len(self.matingPool)-1))
            index_2=math.floor (random.randint (0,len(self.matingPool)-1))
            parent1=self.matingPool[index_1]
            parent2=self.matingPool[index_2]
            child=parent1.crossover(parent2)
            child.mutate(self.mutation_rate)
            self.population[i]=child 

    def fittest(self):
        #returns the fiitest individual mask of the population 
        #also returns the keeping probability of the fittest mask 
        fittest=self.population[0]
        for i  in range (self.popmax):
            if (fittest.fit<self.population[i].fit):
                fittest=self.population[i]
        return fittest,fittest.keep_prob()

In [11]:
class DNA:  
    def __init__(self,maskLength):
        #constructor for the creation of the mask as a gene object 
        self.maskLength=maskLength
        #creation of mask 
        self.gene=torch.bernoulli(torch.empty(1,maskLength).uniform_(0,1))
        self.fit=0

    def keep_prob (self):
        num_one =0
        for i in range (self.maskLength):
            if (self.gene[0,i]==1):
                num_one=num_one+1
        return float(num_one/self.maskLength)

    def fitness(self,model):
        # finding the fitness of a particular mask
        #accuracy of all training set is the fitness in one epoch
        #putting model in train mode 
        running_loss=0
        running_corrects=0
        model.train()

        # for inputs,labels in dataloader ['train']:   ->>>>>>>>>>>>>>changed by riktim
        #     inputs=inputs.to(device)
        #     labels=labels.to(device)
        #     outputs=model.Forward(inputs,self.gene,self.keep_prob())
        #     _,preds=torch.max(outputs,1)
        #     loss=criterion (outputs,labels)
        #     running_loss += loss.item() * inputs.size(0)
        #     running_corrects += torch.sum(preds == labels.data)
        # epoch_loss = running_loss / dataset_sizes['train']
        # epoch_acc = running_corrects.double() / dataset_sizes['train']      
        # self.fit=epoch_acc
        # return epoch_acc
        
        outputs=model.Forward(X_train,self.gene,self.keep_prob())
        _,preds=torch.max(outputs,1)
        loss=criterion (outputs,y_train)
        acc = 100*torch.sum(y_train==torch.max(outputs.data, 1)[1]).double() / len(y_train)
        self.fit=acc
        return acc
        

    def crossover (self,parent2):
        #one parent is the passed in the argument 
        #another parent is the one from which this function is called 
        #another parent is self.gene
        child =DNA(self.maskLength)
        midpoint =random .randint (0,self.maskLength-1)
        for i in range (0,self.maskLength):
            if (i>midpoint):
                child.gene [0,i]=self.gene[0,i]
            else :
                child.gene [0,i]=parent2.gene[0,i]
        
        return child 

    def mutate(self,mutation_rate):
        #randomly activate some of the nodes  
        #mutate some of the genes 
        for i in range (self.maskLength):
            if (random.randint (0,99)<=mutation_rate*100):
                self.gene[0,i]=1 

In [12]:
mutation_rate =0
max_population=30
maskLength=X_train.shape[1]
#seeded so that each time same initial weights generated 
#torch.manual_seed(6)
model=Model(input_size, hidden_size, num_classes)#creating the object of the class
model.to(device)
criterion=nn.CrossEntropyLoss()
#optimizer=torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
#control block controls the epochs and the generations of mask
#step 1 an object of the population class randomly generating the first population 
#step2 :calculate fitness of each entitiy of the population 
#step3: creates a mating pool of the population based on the worst two performing parent 
#step 4 :fittest mask of the generating along with keep_prob found 
#step 5: if 0th ,10th ,20th, the epochs starts training on the worst performing mask /other wise new generation is created 

epochgens=0
population =Population(mutation_rate,max_population,maskLength)
total_acc=[]
p=0
while (epochgens<=num_epochs):
    print ('Epoch generations (',epochgens,'/{})'.format(num_epochs),end=' :')
    if p < 0.8:
      population .calcFitness(model)
      population.naturalSelection()
      fittestmask,p = population .fittest()
      accuracy=fittestmask.fitness(model)
      print ("accuracy(fittest mask)",accuracy,"keep_prob",p,end='\n')
    if (epochgens%10==0):
        model,losses,accuracies=train_model(model,criterion,optimizer,fittestmask.gene,p,30)
        total_acc=total_acc+accuracies
    population.generate()
    epochgens+=1

Epoch generations ( 0 /100) :accuracy(fittest mask) tensor(13.9524, dtype=torch.float64) keep_prob 0.4166666666666667
Epoch 0/29
----------
********************train*************
Epoch [1/30] Loss: 2.5205   Acc: 13.9524
********************val*************
Epoch [1/30] Loss: 2.4841   Acc: 10.6389
Epoch 1/29
----------
********************train*************
Epoch [2/30] Loss: 2.3832   Acc: 19.7024
********************val*************
Epoch [2/30] Loss: 2.4269   Acc: 14.5833
Epoch 2/29
----------
********************train*************
Epoch [3/30] Loss: 2.2571   Acc: 28.3214
********************val*************
Epoch [3/30] Loss: 2.3718   Acc: 17.5000
Epoch 3/29
----------
********************train*************
Epoch [4/30] Loss: 2.1419   Acc: 34.6310
********************val*************
Epoch [4/30] Loss: 2.3187   Acc: 24.4167
Epoch 4/29
----------
********************train*************
Epoch [5/30] Loss: 2.0378   Acc: 39.6905
********************val*************
Epoch [5/30] Loss: 2.26

********************train*************
Epoch [12/30] Loss: 0.8414   Acc: 78.0000
********************val*************
Epoch [12/30] Loss: 0.9714   Acc: 78.7222
Epoch 12/29
----------
********************train*************
Epoch [13/30] Loss: 0.8085   Acc: 78.5119
********************val*************
Epoch [13/30] Loss: 0.9474   Acc: 79.4167
Epoch 13/29
----------
********************train*************
Epoch [14/30] Loss: 0.7775   Acc: 79.3095
********************val*************
Epoch [14/30] Loss: 0.9246   Acc: 80.0000
Epoch 14/29
----------
********************train*************
Epoch [15/30] Loss: 0.7485   Acc: 80.2024
********************val*************
Epoch [15/30] Loss: 0.9028   Acc: 80.3889
Epoch 15/29
----------
********************train*************
Epoch [16/30] Loss: 0.7211   Acc: 80.7262
********************val*************
Epoch [16/30] Loss: 0.8820   Acc: 80.5556
Epoch 16/29
----------
********************train*************
Epoch [17/30] Loss: 0.6955   Acc: 81.2500
****

********************train*************
Epoch [22/30] Loss: 0.3339   Acc: 91.0238
********************val*************
Epoch [22/30] Loss: 0.4675   Acc: 88.0556
Epoch 22/29
----------
********************train*************
Epoch [23/30] Loss: 0.3264   Acc: 91.1786
********************val*************
Epoch [23/30] Loss: 0.4615   Acc: 88.1389
Epoch 23/29
----------
********************train*************
Epoch [24/30] Loss: 0.3192   Acc: 91.5119
********************val*************
Epoch [24/30] Loss: 0.4555   Acc: 88.3333
Epoch 24/29
----------
********************train*************
Epoch [25/30] Loss: 0.3123   Acc: 91.6548
********************val*************
Epoch [25/30] Loss: 0.4495   Acc: 88.3611
Epoch 25/29
----------
********************train*************
Epoch [26/30] Loss: 0.3058   Acc: 91.7619
********************val*************
Epoch [26/30] Loss: 0.4435   Acc: 88.4722
Epoch 26/29
----------
********************train*************
Epoch [27/30] Loss: 0.2995   Acc: 91.9286
****

Epoch generations ( 31 /100) :accuracy(fittest mask) tensor(94.4167, dtype=torch.float64) keep_prob 0.6111111111111112
Epoch generations ( 32 /100) :accuracy(fittest mask) tensor(93.8690, dtype=torch.float64) keep_prob 0.5611111111111111
Epoch generations ( 33 /100) :accuracy(fittest mask) tensor(93.6190, dtype=torch.float64) keep_prob 0.5888888888888889
Epoch generations ( 34 /100) :accuracy(fittest mask) tensor(93.7262, dtype=torch.float64) keep_prob 0.5833333333333334
Epoch generations ( 35 /100) :accuracy(fittest mask) tensor(93.4762, dtype=torch.float64) keep_prob 0.6166666666666667
Epoch generations ( 36 /100) :accuracy(fittest mask) tensor(93.5476, dtype=torch.float64) keep_prob 0.6444444444444445
Epoch generations ( 37 /100) :accuracy(fittest mask) tensor(93.5476, dtype=torch.float64) keep_prob 0.6444444444444445
Epoch generations ( 38 /100) :accuracy(fittest mask) tensor(93.2143, dtype=torch.float64) keep_prob 0.6444444444444445
Epoch generations ( 39 /100) :accuracy(fittest m

********************val*************
Epoch [2/30] Loss: 0.2488   Acc: 93.3889
Epoch 2/29
----------
********************train*************
Epoch [3/30] Loss: 0.1921   Acc: 95.2500
********************val*************
Epoch [3/30] Loss: 0.2445   Acc: 93.6389
Epoch 3/29
----------
********************train*************
Epoch [4/30] Loss: 0.1883   Acc: 95.2857
********************val*************
Epoch [4/30] Loss: 0.2413   Acc: 93.8333
Epoch 4/29
----------
********************train*************
Epoch [5/30] Loss: 0.1853   Acc: 95.4643
********************val*************
Epoch [5/30] Loss: 0.2389   Acc: 93.8889
Epoch 5/29
----------
********************train*************
Epoch [6/30] Loss: 0.1825   Acc: 95.5595
********************val*************
Epoch [6/30] Loss: 0.2372   Acc: 94.0556
Epoch 6/29
----------
********************train*************
Epoch [7/30] Loss: 0.1798   Acc: 95.8929
********************val*************
Epoch [7/30] Loss: 0.2362   Acc: 94.0000
Epoch 7/29
----------


********************train*************
Epoch [12/30] Loss: 0.1237   Acc: 97.2024
********************val*************
Epoch [12/30] Loss: 0.1910   Acc: 95.0278
Epoch 12/29
----------
********************train*************
Epoch [13/30] Loss: 0.1219   Acc: 97.2143
********************val*************
Epoch [13/30] Loss: 0.1925   Acc: 94.9444
Epoch 13/29
----------
********************train*************
Epoch [14/30] Loss: 0.1205   Acc: 97.2619
********************val*************
Epoch [14/30] Loss: 0.1929   Acc: 94.9167
Epoch 14/29
----------
********************train*************
Epoch [15/30] Loss: 0.1192   Acc: 97.3452
********************val*************
Epoch [15/30] Loss: 0.1911   Acc: 94.9167
Epoch 15/29
----------
********************train*************
Epoch [16/30] Loss: 0.1175   Acc: 97.3690
********************val*************
Epoch [16/30] Loss: 0.1874   Acc: 94.9444
Epoch 16/29
----------
********************train*************
Epoch [17/30] Loss: 0.1156   Acc: 97.4167
****

********************train*************
Epoch [22/30] Loss: 0.0808   Acc: 98.2381
********************val*************
Epoch [22/30] Loss: 0.1382   Acc: 96.4722
Epoch 22/29
----------
********************train*************
Epoch [23/30] Loss: 0.0799   Acc: 98.2500
********************val*************
Epoch [23/30] Loss: 0.1368   Acc: 96.5278
Epoch 23/29
----------
********************train*************
Epoch [24/30] Loss: 0.0789   Acc: 98.3095
********************val*************
Epoch [24/30] Loss: 0.1352   Acc: 96.6944
Epoch 24/29
----------
********************train*************
Epoch [25/30] Loss: 0.0781   Acc: 98.2976
********************val*************
Epoch [25/30] Loss: 0.1338   Acc: 96.7222
Epoch 25/29
----------
********************train*************
Epoch [26/30] Loss: 0.0774   Acc: 98.4286
********************val*************
Epoch [26/30] Loss: 0.1326   Acc: 96.7500
Epoch 26/29
----------
********************train*************
Epoch [27/30] Loss: 0.0766   Acc: 98.4762
****

Epoch generations ( 81 /100) :accuracy(fittest mask) tensor(98.9405, dtype=torch.float64) keep_prob 0.7611111111111111
Epoch generations ( 82 /100) :accuracy(fittest mask) tensor(98.6190, dtype=torch.float64) keep_prob 0.7888888888888889
Epoch generations ( 83 /100) :accuracy(fittest mask) tensor(98.4881, dtype=torch.float64) keep_prob 0.7277777777777777
Epoch generations ( 84 /100) :accuracy(fittest mask) tensor(98.4881, dtype=torch.float64) keep_prob 0.7333333333333333
Epoch generations ( 85 /100) :accuracy(fittest mask) tensor(98.5595, dtype=torch.float64) keep_prob 0.7555555555555555
Epoch generations ( 86 /100) :accuracy(fittest mask) tensor(98.5476, dtype=torch.float64) keep_prob 0.7388888888888889
Epoch generations ( 87 /100) :accuracy(fittest mask) tensor(98.5595, dtype=torch.float64) keep_prob 0.7333333333333333
Epoch generations ( 88 /100) :accuracy(fittest mask) tensor(98.5476, dtype=torch.float64) keep_prob 0.7388888888888889
Epoch generations ( 89 /100) :accuracy(fittest m

********************train*************
Epoch [3/30] Loss: 0.0519   Acc: 99.0476
********************val*************
Epoch [3/30] Loss: 0.0973   Acc: 97.3333
Epoch 3/29
----------
********************train*************
Epoch [4/30] Loss: 0.0507   Acc: 99.1190
********************val*************
Epoch [4/30] Loss: 0.0967   Acc: 97.4167
Epoch 4/29
----------
********************train*************
Epoch [5/30] Loss: 0.0499   Acc: 99.1310
********************val*************
Epoch [5/30] Loss: 0.0967   Acc: 97.3889
Epoch 5/29
----------
********************train*************
Epoch [6/30] Loss: 0.0493   Acc: 99.1786
********************val*************
Epoch [6/30] Loss: 0.0972   Acc: 97.4444
Epoch 6/29
----------
********************train*************
Epoch [7/30] Loss: 0.0488   Acc: 99.1786
********************val*************
Epoch [7/30] Loss: 0.0979   Acc: 97.3611
Epoch 7/29
----------
********************train*************
Epoch [8/30] Loss: 0.0483   Acc: 99.1786
********************

In [14]:
0+(31*10)

310